# Chapter 12. Resilient Distributed Datasets(RDDs)
- 앞에까지는 스파크의 구조적 API를 보았고, 보통은 구조적 API의 사용을 권하지만
- 때때로 비즈니스/엔지니어링 문제가 해결되지 않을 수 있고, 이때 스파크의 하위 수준 API인 RDD를 이용할 수 있다

## 하위 수준 API(Low-Level API)란?
- 분산 데이터(RDD)와 분산 공유 변수(브로드캐스트 변수 및 축적기) 두 가지가 있음

### 언제 하위 수준 API를 사용하는지?
1. 상위 레벨 API에서는 찾을 수 없는 기능이 필요할 때. 예컨대 클러스터 전체의 물리적 데이터 배치를 엄격하게 제어할 때(?)
2. RDD를 이용하여 일부 레거시 코드 베이스를 유지 관리 해야 할 때
3. 사용자 지정 공유 변수를 다뤄야 할 때(14챕터)

- 꼭 이런 이유가 아니더라도 모든 스파크 워크로드는 RDD 기반이므로 이해해두면 좋다
- DF 변환을 수행하면 사실 RDD 변환이 안에서는 돌아가는 거임
- 디버깅하거나 복잡한 워크로드 짜는데 도움됨

### 어떻게 하위 수준 API를 사용하는지?
- SparkContext는 하위 수준 API의 entry point
- 스파크 클러스터에서 계산을 수행하는 도구인 SparkSession을 통해 SparkContext에 접근한다(자세한 건 15장)
- `spark.sparkContext`

## RDD에 대하여
- 1.X 버전에서는 기본 API이고, 2.X에서도 사용 가능하지만 일반적이진 않다
- 하지만 내부에서는 다 RDD로 컴파일됨 -> Spark UI도 RDD의 맥락에서 작업 실행을 설명
- 한 줄 요약: RDD는 병렬로 작동할 수 있는 불변의 분할된 레코드 모음
  - 각 record가 스키마로 구조화된 DF와는 다르다!
  - named tuple의 느낌으로 다가오는데?
- RDD의 모든 레코드는 Java/Python 개체 -> 모든 제어는 가능
  - 근데 모든 조작을 다시 만들어야됨(Reinvent the wheel)
  - 최적화도 안되어있음
  - 그니까 가능하면 구조화된 API쓰자 ..
- RDD가 구조화된 데이터 엔진에 안 쓰이니까 앞에서 본 dataset과 유사하다
  - RDD <-> Dataset 변환은 쉬움 -> 두 API 모두 사용해서 각 API의 장단점을 보겠음
  
### Types of RDDs
- 스파크 API 문서를 보면 RDD의 하위 클래스가 많은데
  - 대부분 DF API의 최적화된 실행 계획을 위한 것
  - 일반 유저는 "일반" RDD나 집계를 위한 키 값 RDD 두 가지만 쓸 수 있고 그 둘만 중요
- RDD는 다음 다섯가지 특성으로 구별된다
  1. 파티션 목록
  2. 각 분할의 계산을 위한 함수
  3. 다른 RDD에 대한 종속성 목록
  4. (optional) key-value RDD를 위한 Partitioner(이게 RDD를 쓰는 핵심 이유일 수 있다. 성능-안정성에 도움이 된다:13장 참조)
  5. (optional) 각 분할을 계산할 기본 위치 목록(ex. HDFS 파일의 블록 위치)
- 이러한 속성을 프로그램을 스케쥴하고 실행하는 스파크의 모든 기능에 영향(determine)
- 다른 종류의 RDD는 위의 속성들을 자체적으로 구현(?)
- RDD는 앞에서 보았던 스파크 프로그래밍 패러다임을 그대로 따른다: lazy, action, eager evaluate, 분산 처리 등
  - 즉 DF랑 동일한 방식이라는 것
- 근데 RDD에는 행이 없고, 개별 레코드(Java/Scala/Python 객체)만이 존재
- RDD API는 Java/Scala/Python 모두에서 사용 가능
  - Scala/Java에서는 raw object를 다루는 거랑 성능 차이 없는데, python에서는 성능이 좀 나가리될 수 있다
  - Python RDD를 쓴다는 건 Python UDF를 row-by-row로 하는 거랑 같은 것
  - 데이터를 python 프로세스에 직렬화하고, python에서 돌린 다음 다시 JVM에 직렬화 -> RDD의 오버헤드가 크다
  - 따라서 구조화된 API를 쓰고 꼭 필요할 때만 RDD를 써라

### 언제 RDD를 쓰나
- 매우 특별한 이유가 없으면 쓰지 말라. DF가 훨씬 효율적이고 안정적이며 표현력이 좋다
- 가장 큰 이유는 보통 데이터의 물리적 배포(사용자 지정 데이터 파티셔닝)을 세부적으로 제어 하는 것

### Datasets and RDDs of Case Classes
- 웹에서 본 건데 흥미로운 질문: Case Calsses의 RDD와 Dataset의 차이는?
  - Dataset은 구조화된 API가 제공하는 기능과 최적화를 활용할 수 있다.
  - Dataset을 쓸 때는 JVM 이나 Spark에서 돌아가는 것 중 하나를 선택할 필요가 없다(?)
  - 가장 쉬운 걸 쓰면 된다

## RDD 만들기

### DF, Dataset, RDD 간의 상호 운용
- RDD를 만드는 가장 쉬운 방법은 기존 DF나 Dataset을 변환하는 것
  - `spark.range(500).rdd`처럼.
- Python에서는 Dataset이 없어서 row형식의 RDD를 반환받게 된다
  - Scala: `spark.range(10).toDF().rdd.map(rowObject => rowObject.getLong(0))`
  - Python: `spark.range(10).toDF("id").rdd.map(lambda row: row[0])`
- 원복은 toDF
  - `spark.range(10).rdd.toDF()`
- 이렇게 만들어진 RDD는 Row 유형.
  - 이 행들은 Spark가 구조화된 API의 데이터를 나타내는데 사용하는 내부 카탈리스트 형식.
  - 이런 식으로 구조화된 API와 하위 수준 API를 왔다갔다
- RDD랑 Dataset은 API가 비슷해보일 거임
  - 실제로 RDD는 Dataset의 하위 수준 형상
  - 둘 다 구조화된 API이 갖고 있는 편리한 기능과 인터페이스가 없음 ...

### 로컬 콜렉션에서
- collection에서 RDD를 만들려면 병렬화를 해야 함
  - 여기서 말하는 collection은 걍 array를 말하는듯?
- 이때 파티션 수를 명시할 수 있음
- 이름도 따로 지어줄 수 있음(Spark UI에서 확인 가능)
```
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")
words = spark.sparkContext.parallelize(myCollection, 2)
```

### Data source에서
- 데이터 소스나 텍스트 파일에서 RDD를 만들 수도 있는데, Data Source API를 사용하는 게 더 좋음
- RDD에는 DF와 달리 데이터 소스 API 개념이 없다
  - 주로 종속 구조와 파티션 목록을 정의
  - 9장에서 본 Data Source API가 거의 항상 더 좋은 방법
- 결과적으로 RDD의 각 레코드가 읽으려던 텍스트 파일 또는 파일의 행을 나타냄 (혹은 두번째 코드처럼 통째로 파일 하나)
- 이 RDD에서 파일의 이름은 첫 번째 개체이고, 텍스트 파일의 값이 두번째 문자열 개체
- `spark.sparkContext.textFile("/some/path/withTextFiles")`
- `spark.sparkContext.wholeTextFiles("/some/path/withTextFiles")`

## RDD 다루기
- RDD를 다루는 건 DF랑 따로 차이가 없다
- 다만 Raw Java/Scala를 개체를 다룬다는 게 핵심 차이

## 변환
- 대부분의 변환은 구조화된 API의 기능을 반영mirror
- DF/Dataset에서처럼 하나의 RDD에서 변환하면 새로운 RDD가 생성됨 -> 종속성도 정의되겠지?

### distinct
- `words.distinct().count()``
- drop_duplicates지 뭐

### filter
- 필터링은 SQL에서 where 절이랑 같은 것
- 필터함수에서는 부울을 반환하는 형식

```
def startsWithS(individual):
    return individual.startswith("S")

words.filter(lambda word: startsWithS(word)).collect()
```

### map
- 11장에서 본 거랑 같은 것
- 값이 주어지면 원하는 걸 반환하는 형식
```
words2 = words.map(lambda word: (word, word[0], word.startswith("S")))
>> “Spark,” “S,” and “true,”
words2.filter(lambda record: record[2]).take(5)
>> Boolean만
```

### flatMap
- map 함수의 간단한 확장: flattening해주는 것
`words.flatMap(lambda word: list(word)).take(5)`


### sort
- sordBy 메소드로 RDD를 정렬 가능
- RDD 개체에서 기준이 되는 값을 추출하고 이걸로 정렬
- 아래 함수는 단어 길이를 기준으로 내림차
`words.sortBy(lambda word: len(word) * -1).take(2)`


### Ransdom Splits
- RDD를 RDD 배열로 무작위 분할 가능
`fiftyFiftySplit = words.randomSplit([0.5, 0.5])`

## Actions
- DF와 Dataset처럼 action을 지정 가능

### reduce
- reduce를 이용하여 모든 종류의 값의 RDD를 하나의 값으로 reduce할 수 있다
- 예를 들어 숫자는 합으로 줄일 수 있다
- 함수형 프로그래밍을 좀 친다면, 새로운 개념new condcept이 되면 안된다?

`spark.sparkContext.parallelize(range(1, 21)).reduce(lambda x, y: x + y) # 210`
- 텍스트에서 가장 긴 놈을 뽑을 수도 있다

```
  def wordLengthReducer(leftWord, rightWord):
    if len(leftWord) > len(rightWord):
      return leftWord
    else:
      return rightWord
  words.reduce(wordLengthReducer)
```
- 파티션에 대한 reduce가 결정론적이지 않아 무조건 왼쪽으로 한다거나 할 수 있다
  - 해결이 안되나 ..?
  
### count
**countApprox**
- 반환 signature(?)이 이상하긴 하지만, 꽤 정교하다
- 근사이긴 한데 빠르고 시간 제한에 따라 결과가 불안정할 수 있다
- 오차 한계가 함께 나온다

```
## 스칼라만 있네 ;;
val confidence = 0.95
val timeoutMilliseconds = 400
words.countApprox(timeoutMilliseconds, confidence)
```

**countApproxDistinct**
- 두 가지 구현이 있고, 둘 다 streamlib의 "Hyperloglog 실전: ..."에서 가져왔음
- 방법 1.
   - 상대적 정확도(>0.000017)를 인수로 받아서, 그 값이 작을수록 더 많은 공간이 필요함
   - `words.countApproxDistinct(0.05)`
- 방법 2. 
  - 일반 데이터의 매개 변수(p)와 희소 표현의 매개 변수(sp)를 인자로 받는다 (모두 정수)
  - 상대적 정확도는 대략 1.054 / sqrt(p)
  - sp > p로 하면 카디널리티가 작을 때 메모리 소비 줄이고 정확도 업업
  - `words.countApproxDistinct(4, 10)`

**countByValue**
- 지정된 RDD의 값의 수를 센다
- 최종적으로 결과 집합을 드라이버의 메모리에 로드
  - 따라서 결과 map이 작을 때만 써야 함
- `words.countByValue()`

**countByValueApprox**
- 위의 것을 근사치로. 
- `words.countByValueApprox(1000, 0.95)`

**first**
- `words.first()`

**max in min**
- `spark.sparkContext.parallelize(1 to 20).max()`
- `spark.sparkContext.parallelize(1 to 20).min()`

**take**
- take랑 그 비슷한 놈들은 RDD에서 여러 값을 가져온다
- 먼저 한 파티션을 스캔하고, 다음 파티션을 스캔해서 필요한 파티션 수를 추정
- takeOrdered, takeSample, top 등이 있다
  - takeSample을 이용하면 RDD에서 고정 크기 랜덤 표본을 가져올 수 있다 (?)
  - top은 takeOrdered와 반대

```
words.take(5)
words.takeOrdered(5)
words.top(5)
val withReplacement = true
val numberToTake = 6
val randomSeed = 100L
words.takeSample(withReplacement, numberToTake, randomSeed)
```

## Saving Files
- 일반 텍스트 파일에 쓰는 것
- RDD를 쓸 때는 Data source를 실제로 "저장"할 수 없다
- 파티션 내용을 하나하나 써야 하고, 상위 레벨 API에서도 똑같다

### saveAsTextFile
- `words.saveAsTextFile("file:/tmp/bookTitle")`
- 압축 코덱을 지정하고 싶으면 하둡에서 임포트해오면 된다

```
import org.apache.hadoop.io.compress.BZip2Codec
words.saveAsTextFile("file:/tmp/bookTitleCompressed", classOf[BZip2Codec])
```

### SequenceFiles
- Spark는 하둡 에코 시스템에서 성장해서 좀 친하다
- SequenceFile은 이진 키-값으로 구성된 플랫 파일
  - MapReduce에서 입/출력 형식으로 광범위하게 사용
- `words.saveAsObjectFile("/tmp/my/sequenceFilePath")`


### Hadoop Files
- 다양한 종류의 하둡 파일 형식이 있다
- 하둡에서 고이거나 레거시 아님 거의 관련 없음 (그래서 생략하는듯)

## Caching
- DF, Data source와 마찬가지로 RDD캐싱에도 동일한 원리 적용됨: RDD 캐시하거나 유지 가능
- 기본적으로 캐시 및 지속의 메모리의 데이터만 처리
- 앞에서 지정했던 이름으로 지정 가능
- `words.cache()``
- singleton 개체의 스토리지 수준(org.apache.spark.storage)으로 스토리지 수준을 지정할 수 있습니다. 저장소 수준 - 메모리만 조합하고 디스크만 조합하며 별도로 힙을 벗어납니다. (이해 못함)

## Checkpointing
- DF에서 쓸 수 없었던 것
- RDD를 디스크에 저장해서 향후 RDD에 대한 참조를 디스크에서 가져가는 것
- 메모리 말고 디스크에 쓰지만 개념적으로는 캐싱과 유사

```
spark.sparkContext.setCheckpointDir("/some/path/for/checkpointing")
words.checkpoint()
```

## Pipe RDDs to System Commands
- 외부에 ETL을 붙인다는 건가?
- 파이프를 사용하면 파이프 요소에 의해 생성된 RDD를 외부 프로세스에 반환 가능
- RDD는 파티션 당 한 번씩 지정된 프로세스를 실행/계산?

### mapPartitions
- 잘 이해안감
- 그니까 스파크는 파티션해서 돌린다는 거고
- RDD에 대한 map 작업이 까보면 MapPartitionRDD라는 거다
- `words.mapPartitions(lambda part: [1]).sum() // 2`
- 각 row에 대해 하는 것 같지만, 클러스터가 물리적으로 데이터를 운영하는 단위는 파티션이니까

```
def indexedFunc(partitionIndex, withinPartIterator):
    return ["partition: {} => {}".format(partitionIndex,
      x) for x in withinPartIterator]
  words.mapPartitionsWithIndex(indexedFunc).collect()
```

### foreachPartition
- 반환없이 돌아간다?
- 반환없이 DB에 쓴다?
- 많은 data source커넥터가 쓰이는 방법이다 ..?
- 임시로 파일을 떨굴 수도 있다(예제 코드처럼)

```
words.foreachPartition { iter =>
   import java.io._
   import scala.util.Random
   val randomFileName = new Random().nextInt()
   val pw = new PrintWriter(new File(s"/tmp/random-file-${randomFileName}.txt"))
   while (iter.hasNext) {
       pw.write(iter.next())
    }
pw.close() }
```

### glom
- Dataset의 모든 파티션을 가져와 배열로 변환한
- 드라이버에 데이터를 수집하고 각 파티션에 배열을 사용할 때 유용
- 파티션이 크거나 수가 많으면 드라이버가 충돌 -> 안정성 문제
- `spark.sparkContext.parallelize(["Hello", "World"], 2).glom().collect()`

## Conclusion
- 단일 RDD와 RDD API의 기본 사항들을 봤다
- 담 장에서는 join 및 key-value RDD와 같은 고오급 RDD를 보겠다